In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import gym

sys.path.insert(0, './TetrisEnv')
from TetrisBattle.envs.tetris_env import TetrisSingleEnv, TetrisEnv
from TetrisBattle import *
from Model.Feature_Extraction import TetrisActorCriticCnnPolicyLookAhead
# %pip install -e ./TetrisEnv
# import TetrisBattle

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
class InputGetter():
    def __init__(self):
        self.input_log = []
        
    def get_input(self):
        while True:
            try:
                if len(self.input_log) == 0:
                    self.input_log = [*input(">")]
                
                inp = self.input_log.pop(0)
                inp = int(inp)
                assert(inp in [0, 1, 2, 3, 4, 5, 6, 7, 9])
                return inp
            except Exception as e:
                print(e)

In [ ]:
env = TetrisSingleEnv(obs_type="lookahead")
env.reset()
env.render()
i = 0
inputter = InputGetter()
while True:
    action = inputter.get_input()
    if action == 9:
        env.close()
        break
    state, reward, done, info = env.step(env.action_space.sample())
    # print("infow", info)
    # print("rewrd",reward)
    env.render()
    print(state.shape)
    print(list(state))
    # print(env.game_interface.get_obs().shape)
    grid = env.game_interface.get_obs()

In [6]:
state.shape

(10, 880, 1)

In [ ]:
from Model.Feature_Extraction import TetisFeatureExtractorLookAhead
fe = TetisFeatureExtractorLookAhead(env.observation_space)
# fe(grid[None])

In [ ]:
env = TetrisSingleEnv()
i = 0
inputter = InputGetter()
while True:
    action = inputter.get_input()
    if action == 9:
        env.close()
        break
    state, reward, done, info = env.og_step(action)
    print("infow", info)
    print("rewrd",reward)
    env.render()
    print(env.get_obs())

In [ ]:

# env = make('SinglePTetris-v0')
env = TetrisSingleEnv()
ob = env.reset()
dots = np.ones((20, 10))
x = 100
y = 130
ob[x:x+10,y:y+10] = [255,255,255]
plt.imshow(ob, cmap="plasma")


In [ ]:
from shimmy.openai_gym_compatibility import _convert_space
lh_env = TetrisSingleEnv(obs_type="lookahead")
g_env = TetrisSingleEnv(obs_type="grid")

gs = g_env.reset()
ls = lh_env.reset()
print(gs.shape, type(gs))
print(ls.shape, type(ls))
# # _convert_space(g_env.observation_space)
# # _convert_space(lh_env.observation_space)
# # print(g_env.observation_space)
# print(lh_env.observation_space)

In [3]:
from gym.wrappers import GrayScaleObservation
from preprocessing.EensyWeensy import MakeEensyWeensy
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# from gym import make

naked_env = TetrisSingleEnv(obs_type="lookahead")
# env = GrayScaleObservation(env, keep_dim=True)
# env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = DummyVecEnv([lambda: naked_env])
# env = VecFrameStack(env, 4, channels_order="last")

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
/opt/miniconda3/envs/ECE_113_Capstone_1/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: Y

In [ ]:
# from Model.Feature_Extraction import TetisFeatureExtractor
# feat_ex = TetisFeatureExtractor(gym.spaces.Box(0.0, 1.0, (20, 34), "float32"))

In [ ]:
ob,_,_,_ = env.step([0])
ob.transpose(3, 0, 1, 2).shape

In [ ]:
# for i in range(0, 1000):
#     action = env.random_action()
#     state, reward, done, info = env.step(action)
#     env.render()
#     i += 1
# env.close()

In [4]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
import torch
print(torch.cuda.is_available())
# check_env(TetrisSingleEnv())

False


In [5]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [7]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [10]:
try:
    del model
except NameError:
    pass
model = PPO(TetrisActorCriticCnnPolicyLookAhead, env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=1e-2, n_steps=32, device="cuda") 


Using cpu device


/opt/miniconda3/envs/ECE_113_Capstone_1/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 32`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 32
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=32 and n_envs=1)
  warnings.warn(


In [11]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22       |
|    ep_rew_mean     | -96.3    |
| time/              |          |
|    fps             | 8        |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 32       |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 21         |
|    ep_rew_mean          | -102       |
| time/                   |            |
|    fps                  | 5          |
|    iterations           | 2          |
|    time_elapsed         | 10         |
|    total_timesteps      | 64         |
| train/                  |            |
|    approx_kl            | 0.40507835 |
|    clip_fraction        | 0.769      |
|    clip_range           | 0.2        |
|    entropy_loss         | -3.69      |
|    explained_variance   | 0.0116     |
|    learning_rate   

In [ ]:
# model = PPO.load('./train/best_model_20000')
test_env = TetrisSingleEnv(obs_type="grid")
state = test_env.reset()
i = 0

while True:
    if (i >= 1000):
        env.close()
        break
    action, _ = model.predict(state)
    state, reward, done, info = test_env.step(action)
    print("reward", reward)
    print(done)
    test_env.render()
    i += 1

In [ ]:
state = naked_env.reset()
print(state)
print(model.predict(state))